# Noise Analysis
Import our standard tools

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch # power spectral density fxn.
from time import sleep
import time, os
import datetime

Now, we will import our udp tools from udpcap.py. We'll save the data to a timestamped file and extract our I/Q data.

In [2]:
import udpcap
cap = udpcap.udpcap()

192.168.3.40
4096


In [3]:
# Create our connection 
cap.bindSocket()

In [5]:
# Create a timestamped filename, capture packets, then save to a file
fname = 'udpData{0:%Y%m%d%H%M%S}'.format(datetime.datetime.now())
npoints = 1024
data = cap.capture_packets(npoints)
np.save(fname, data)

KeyboardInterrupt: 

In [6]:
# release connection
cap.release()

In [ ]:
I=data[::2]
Q=data[1::2]


In [ ]:
len(I.T[0])

In [ ]:
# view I/Q magnitude
Iacc_f = I.T[0][:]/2**10#.astype("float")
Qacc_f = Q.T[0][:]/2**10#.astype("float")
Ia, Qa = np.array(Iacc_f).astype(float), np.array(Qacc_f).astype(float)
IQMAG=np.sqrt(Ia**2+Qa**2)


plt.figure(figsize=(18,12))
#plt.xlim(0,150)
plt.plot(IQMAG[0:1024][:],"-+")
#plt.ylim(0,60)


np.argmax(IQMAG)

In [ ]:
plt.figure(figsize=(18,12))
#plt.xlim(0,150)
plt.plot(IQMAG[0:1024][:],"-+")
#plt.ylim(0,60)

In [ ]:
np.argmax(abs(I[:,0]))

In [ ]:
c=33# data index with channel data
f,Spp=welch(np.arctan2(Q[c,:],I[c,:]),fs=512e6/2**(10+10),nperseg=1024)
plt.figure()
plt.plot(Q[c,:]-np.mean(Q[c,:]))
plt.figure()
plt.plot(I[c,:]-np.mean(I[c,:]))
print(np.mean(I[c,:]))
print(np.mean(Q[c,:]))
plt.figure()
plt.semilogx(f[2:],10*np.log10(Spp)[2:],"-+",label="10-tone")

In [ ]:
# create detector-detector correlation matrix
Z = I + 1j*Q
deproj_I = I
deproj_Q = Q
correlation_matrix = np.matmul(Z,np.conj(np.transpose(Z)))
# calculate the eigenmodes of the correlation matrix
w,v = np.linalg.eig(correlation_matrix)
print(w)
# create a template based on the largest eigenmode
template = np.matmul(v[:,0],Z)
template1 = np.matmul(v[:,1],Z)

template_real = np.real(template)-np.mean(np.real(template))
template_imag = np.imag(template)-np.mean(np.imag(template))
template_z = template_real + 1j*template_imag

template1_real = np.real(template1)-np.mean(np.real(template1))
template1_imag = np.imag(template1)-np.mean(np.imag(template1))
template1_z = template1_real + 1j*template1_imag

# rotate so that the signal is in one quadrature
template_corr = np.matmul(template_imag,np.transpose(template_real))/np.matmul(template_real,np.transpose(template_real))
template_imag_new = template_imag - template_real*template_corr
template_real_new = (template_imag + template_imag*template_corr)/2
template_real = template_real_new
template_imag = template_imag_new

# rotate so that the signal is in one quadrature
#template1_corr = np.matmul(template1_imag,np.transpose(template1_real))/np.matmul(template1_real,np.transpose(template1_real))
#template1_imag_new = template1_imag - template1_real*template1_corr
#template1_real_new = (template1_imag + template1_real*template1_corr)/2
#template1_real = template1_real_new
#template1_imag = template1_imag_new

# plot the templates

plt.figure()
plt.plot(template_real,label='I')
plt.plot(template_imag,label='Q')
plt.legend()

plt.figure()
plt.plot(template1_real,label='I')
#plt.plot(template1_imag,label='Q')
plt.legend()
plt.figure()
plt.plot(template1_imag,label='Q')
#plt.plot(template1_imag,label='Q')
plt.legend()
# based on looking at the template, define the imaginary part to be zero
#template = np.real(template) + 0.0j

In [ ]:
c = 35
# get the complex signal from the current detector
complex_detector_signal = Z[c,:]
# calculate the dot product with the template
#corr_with_template = np.matmul(complex_detector_signal,np.transpose(template_z))/np.matmul(template_z,np.conj(np.transpose(template_z)))
corr_with_real_i = np.matmul(I[c,:],np.transpose(template_real))/np.matmul(template_real,np.transpose(template_real))
print(corr_with_real_i)
corr_with_real_q = np.matmul(Q[c,:],np.transpose(template_real))/np.matmul(template_real,np.transpose(template_real))
print(corr_with_real_q)
# remove the template from the signal
corr_with_imag_i = np.matmul(I[c,:],np.transpose(template_imag))/np.matmul(template_imag,np.transpose(template_imag))
print(corr_with_imag_i)
corr_with_imag_q = np.matmul(Q[c,:],np.transpose(template_imag))/np.matmul(template_imag,np.transpose(template_imag))
print(corr_with_imag_q)
# remove the template from the signal
deprojected_detector_i = I[c,:]-corr_with_real_i*template_real#-corr_with_imag_i*template_imag
deprojected_detector_q = Q[c,:]-corr_with_real_q*template_real#-corr_with_imag_q*template_imag

complex_deprojected_detector_signal = deprojected_detector_i + 1j*deprojected_detector_q
#complex_deprojected_detector_signal = complex_detector_signal - corr_with_template*template_z
# plot I and Q for both
plt.figure()

plt.plot(I[c,:] - np.mean(I[c,:]),label='Raw Detector #'+str(c)+' Raw I Signal')
plt.plot(deprojected_detector_i - np.mean(deprojected_detector_i),label='Template Deprojected')

#plt.plot(np.real(complex_detector_signal) - np.mean(np.real(complex_detector_signal)),label='Raw Detector #'+str(c)+' Phase Signal')
#plt.plot(np.real(complex_deprojected_detector_signal) - np.mean(np.real(complex_deprojected_detector_signal)),label='Template Deprojected')
plt.legend()
plt.ylabel('I')
plt.figure()

plt.plot(Q[c,:] - np.mean(Q[c,:]),label='Raw Detector #'+str(c)+' Raw Q Signal')
plt.plot(deprojected_detector_q - np.mean(deprojected_detector_q),label='Template Deprojected')

#plt.plot(np.imag(complex_detector_signal) - np.mean(np.imag(complex_detector_signal)),label='Raw Detector #'+str(c)+' Phase Signal')
#plt.plot(np.imag(complex_deprojected_detector_signal) - np.mean(np.imag(complex_deprojected_detector_signal)),label='Template Deprojected')
plt.legend()
plt.ylabel('Q')

f,Spp_detector_signal=welch(np.arctan2(np.imag(complex_detector_signal),np.real(complex_detector_signal)),fs=512e6/2**(10+10),nperseg=npoints)
f,Spp_deprojected_detector_signal=welch(np.arctan2(np.imag(complex_deprojected_detector_signal),np.real(complex_deprojected_detector_signal)),fs=512e6/2**(10+10),nperseg=npoints)

#f,Spp_detector_signal=welch(np.arctan2(np.imag(complex_detector_signal),np.real(complex_detector_signal)),fs=512e6/2**(10+10),nperseg=1024)
#f,Spp_deprojected_detector_signal=welch(np.arctan2(np.imag(complex_deprojected_detector_signal),np.real(complex_deprojected_detector_signal)),fs=512e6/2**(10+10),nperseg=1024)
plt.figure()
plt.semilogx(f[2:],10*np.log10(Spp_detector_signal)[2:],"-+",label="Raw Signal")
plt.semilogx(f[2:],10*np.log10(Spp_deprojected_detector_signal)[2:],"-+",label="Deprojected")
plt.legend()

norm = np.mean(np.abs(complex_deprojected_detector_signal))
print(norm)
f,Spp_detector_signal_i=welch(np.real(complex_detector_signal)/norm,fs=512e6/2**(10+10),nperseg=npoints)
f,Spp_deprojected_detector_signal_i=welch(np.real(complex_deprojected_detector_signal)/norm,fs=512e6/2**(10+10),nperseg=npoints)

#f,Spp_detector_signal=welch(np.arctan2(np.imag(complex_detector_signal),np.real(complex_detector_signal)),fs=512e6/2**(10+10),nperseg=1024)
#f,Spp_deprojected_detector_signal=welch(np.arctan2(np.imag(complex_deprojected_detector_signal),np.real(complex_deprojected_detector_signal)),fs=512e6/2**(10+10),nperseg=1024)
plt.figure()
plt.semilogx(f[2:],10*np.log10(Spp_detector_signal_i)[2:],"-+",label="Raw Signal I")
plt.semilogx(f[2:],10*np.log10(Spp_deprojected_detector_signal_i)[2:],"-+",label="Deprojected I")
plt.legend()
print(np.mean(10*np.log10(Spp_deprojected_detector_signal_i)[2:]))

In [ ]:
chnls = range(0,128)
for i in chnls:
    sleep(1)
    f,Spp=welch(np.arctan2(Q[i,:],I[i,:]),fs=512e6/2**20,nperseg=npoints)
    l=len(Spp)
    if np.mean(Spp[l//7:])>1e-7:
#     if(1==1):
        plt.figure(figsize=(16,9))

        plt.subplot(231)
        plt.ylabel(f"i={i}",fontsize=16)
        plt.plot(I[i,:],color="green"); 
        plt.plot(Q[i,:],color="blue"); # +/-1e7

        plt.subplot(232)
        plt.plot(I[i,:],color="green"); 

        plt.subplot(233)
        plt.plot(Q[i,:],color="blue"); # +/-1e7
        
        plt.subplot(2,3,(4,6) )
       
        plt.loglog(f,Spp,color="firebrick",alpha=1,marker="+"); plt.ylim(1e-7,1e-1);
        plt.ylabel(r"$S_{\phi \phi}$ [dBc/Hz]",fontsize=12)
        plt.xlabel("[Hz]",fontsize=12)
        plt.grid()

        plt.axhline(np.mean(Spp[l//7:]),color="black")
        plt.axvspan(f[l//7], f[-1], alpha=0.5, color='grey')
#         plt.savefig(f"./figs/chnl-{i}_{fname}")       

In [ ]:
chnls = range(0,256)
plt.figure(figsize=(11,12))

#plt.subplot(211)
plt.title("100-tones-2 (-200,200)")
#plt.plot(I[c,0])
#plt.plot(Q[c,0])
#plt.subplot(212)
nfs = np.int(np.log2(npoints))
f_interp = np.zeros(nfs)
Spp_i_interp = np.zeros(nfs)
Spp_q_interp = np.zeros(nfs)
Smean=[]
norms=[]
for chnl in chnls:
    complex_detector_signal = Z[chnl,:]
    if np.mean(np.abs(complex_detector_signal)) > 100:
# calculate the dot product with the template
#corr_with_template = np.matmul(complex_detector_signal,np.transpose(template_z))/np.matmul(template_z,np.conj(np.transpose(template_z)))
        corr_with_real_i = np.matmul(I[chnl,:],np.transpose(template_real))/np.matmul(template_real,np.transpose(template_real))
        print(corr_with_real_i)
        corr_with_real_q = np.matmul(Q[chnl,:],np.transpose(template_real))/np.matmul(template_real,np.transpose(template_real))
        print(corr_with_real_q)
        corr_with_imag_i = np.matmul(I[chnl,:],np.transpose(template_imag))/np.matmul(template_imag,np.transpose(template_imag))
        print(corr_with_imag_i)
        corr_with_imag_q = np.matmul(Q[chnl,:],np.transpose(template_imag))/np.matmul(template_imag,np.transpose(template_imag))
        print(corr_with_imag_q)
# remove the template from the signal
        deprojected_detector_i = I[chnl,:]-corr_with_real_i*template_real#-corr_with_imag_i*template_imag
        deprojected_detector_q = Q[chnl,:]-corr_with_real_q*template_real#-corr_with_imag_q*template_imag

        complex_deprojected_detector_signal = deprojected_detector_i + 1j*deprojected_detector_q
        norm = np.mean(np.abs(complex_deprojected_detector_signal))
        print(norm)
        norms.append(norm)

#print(norm)
#f,Spp_detector_signal_i=welch(np.real(complex_detector_signal)/norm,fs=512e6/2**(10+10),nperseg=npoints)
        f,Spp_i=welch(np.real(complex_deprojected_detector_signal)/norm,fs=512e6/2**(10+10),nperseg=npoints)
        f,Spp_q=welch(np.imag(complex_deprojected_detector_signal)/norm,fs=512e6/2**(10+10),nperseg=npoints)

        
        f,Spp=welch(np.arctan2(Q[chnl,:],I[chnl,:]),fs=512e6/2**20,nperseg=npoints)
        #plt.semilogx(f[2:],10*np.log10(Spp_i)[2:],"-+",color="cyan",alpha=0.2,label="Deprojected I")
        #plt.semilogx(f[2:],10*np.log10(Spp_q)[2:],"-+",color="orange",alpha=0.2,label="Deprojected Q")

        for j in range(0,nfs-1):
            f_interp[j] = np.mean(f[2**j:2**(j+1)])
            Spp_i_interp[j] = np.mean(Spp_i[2**j:2**(j+1)])
            Spp_q_interp[j] = np.mean(Spp_q[2**j:2**(j+1)])
        plt.semilogx(f_interp[:],10*np.log10(Spp_i_interp)[:],"-+",color="cyan",alpha=0.4,label="Deprojected I")
        plt.semilogx(f_interp[:],10*np.log10(Spp_q_interp)[:],"-+",color="orange",alpha=0.4,label="Deprojected Q")

        #plt.loglog(f,Spp_i,color="firebrick",alpha=.40)
        Smean.append(np.mean(np.log10(Spp_i)))

plt.ylabel(r"$S_{\phi \phi}$ [dBc/Hz]", fontsize=16); 
plt.xlabel("[MHz]", fontsize=16)
plt.ylim(-150,-50)
plt.grid()

# Export plot to folder and display
fname = 'noisePlot{0:%Y%m%d%H%M%S}.png'.format(datetime.datetime.now())
plt.savefig(fname)
plt.show()

plt.plot(norms)
plt.show
# plt.hist(Smean,bins=10)
# plt.show()
#plt.plot(Svar)
#plt.show()

In [ ]:
f_interp = np.array(nfs)
print(f_interp)
print(np.mean(f[2**j:2**(j+1)]))

In [ ]:
print(np.int(np.log2(npoints)))